# Example of a D-efficient RUM design with dummies in ChoiceDesign

This notebook illustrates how to use **ChoiceDesign** to generate a D-efficient experimental design with some attributes coded as dummies. Given a set of attributes and prior parameters, ChoiceDesign uses a variation of the random swapping algorithm [1] to minimise the D-error of the information matrix of a Multinomial Logit (MNL) model.

## Step 1: Load modules, define design parameters and set attributes

The following lines load:
- `EffDesign`: the class of efficient designs,
- `Attribute` and `Parameter`: the classes of attributes and parameters, respectively.

In [25]:
from choicedesign.design import EffDesign
from choicedesign.expressions import Attribute, Parameter

Each attribute is defined by the `Attribute` class. The arguments of this class are:

* `name`: a string with the attribute name,
* `levels`: a list of levels of the attribute,

Each attribute is alternative-specific. Hence, attributes must be defined for each alternative that contains them.

The following lines define 2 alternatives, named `alt1` and `alt2`, and 4 attributes named from $A$ to $D$:

In [26]:
alt1_A = Attribute('alt1_A',[1,2,3])
alt1_B = Attribute('alt1_B',[1,2,3])
alt1_C = Attribute('alt1_C',[0,3,5])
alt1_D = Attribute('alt1_D',[0,1,2])

alt2_A = Attribute('alt2_A',[1,2,3])
alt2_B = Attribute('alt2_B',[1,2,3])
alt2_C = Attribute('alt2_C',[0,3,5])
alt2_D = Attribute('alt2_D',[0,1,2])

## Step 2: Construct efficient design object and generate initial design matrix

The second step consists of constructing the experimental design object, which requires the following parameters:

- `X`: A list of `Attribute` class elements,
- `ncs`: The number of choice situations.

The following lines define a object named `design` using `EffDesign` of 16 choice situations:

In [27]:
design = EffDesign(
    X = [alt1_A,alt1_B,alt1_C,alt1_D,
         alt2_A,alt2_B,alt2_C,alt2_D],
    ncs=16)

After the design object is defined, the method `gen_initdesign()` generates the initial design matrix. This method accepts the following optional parameters:

* `cond`: List of conditions that the final design must hold. Each element is a string that contains a single condition. Conditions can be of the form of binary relations (e.g., `X > Y` where `X` and `Y` are attributes of a specific alternative) or conditional relations (e.g., `if X > a then Y < b` where `a` and `b` are values). Users can specify multiple conditions when the operator `if` is defined, separated by the operator `&`.

* `seed`: Random seed

For this example, neither of the arguments above will be used:

In [28]:
init_design = design.gen_initdesign()
init_design

,alt1_A,alt1_B,alt1_C,alt1_D,alt2_A,alt2_B,alt2_C,alt2_D
0,2,1,5,1,1,3,5,1
1,2,2,0,1,2,1,5,0
2,1,1,5,0,2,2,5,2
3,2,1,3,2,2,3,0,0
4,1,3,3,0,1,2,0,2
5,2,2,3,1,1,2,5,2
6,1,3,5,2,3,3,5,1
7,2,2,5,2,3,1,3,0
8,3,1,0,1,1,1,0,2
9,3,1,5,0,1,1,0,1


## Step 3: Set the utility functions

`ChoiceDesign` allows users to customise the utility functions of the efficient design. The syntax is based as in Biogeme: each utility function contains attributes and parameters. For this, we use the `Parameter` class, which requires the following arguments:

* `name`: The parameter name
* `value`: The prior value

We will assume that attributes `A` and `B` are coded as dummies in which level 1 is the baseline. Therefore, we must define additional parameters:

The following lines define six parameters:

In [29]:
beta_A_2 = Parameter('beta_A_2',-0.1)
beta_A_3 = Parameter('beta_A_3',-0.4)

beta_B_2 = Parameter('beta_B_2',-0.02)
beta_B_3 = Parameter('beta_B_3',-0.01)

beta_C = Parameter('beta_C',0.1)
beta_D = Parameter('beta_D',0.15)


Then, the utility functions must be defined. We will asume a linear utility function for each alternative.

In [30]:
V1 = beta_A_2 * (alt1_A==2) + beta_A_3 * (alt1_A==3) + beta_B_2 * (alt1_B==2) + beta_B_3 * (alt1_B==3) + beta_C * alt1_C + beta_D * alt1_D
V2 = beta_A_2 * (alt2_A==2) + beta_A_3 * (alt2_A==3) + beta_B_2 * (alt2_B==2) + beta_B_3 * (alt2_B==3) + beta_C * alt2_C + beta_D * alt2_D

The utility functions must be stored in a dictionary object. In this dictionary, each key is a consecutive number from 1 to the number of alternatves. The values of each key are the corresponding utility functions:

In [31]:
V = {1: V1, 2: V2}

## Step 3: Optimise the initial design, given the utility functions and priors:

The method `optimise()` starts the D-error minimisation routine, given the initial design matrix and the utility functions. This method requires the following parameters:

* `init_design`: The objective design matrix to optimise
* `V`: The dictionary object with utility functions
* `model`: The base model of the efficient design. By default is `mnl` for a Multinomial Logit model.

In addition, `optimise()` admits the following optimal parameters:

* `n_blocks`: number of blocks of the final design. Must be a multiple of the number of choice situations,
* `iter_lim`: number of iterations before the algorithm stops.
* `noimprov_lim`: Number of iterations without improvement before the algorithm stops,
* `time_lim`: time (in minutes) before the algorithm stops,
* `seed`: Random seed
* `verbose`:Whether status messages and progress are shown.

The outputs of `optimise` are:

* `optimal_design`: The optimised design matrix
* `init_perf`: The initial D-Error
* `final_perf`: The D-error of the last stored design
* `final_iter`: The last iteration number
* `ubalance_ratio`: The utility balance ratio. a 0% value indicates strict dominance of an alternative, whereas 100% indicates equal market shares. 

The following line starts the optimisation routine during 1 minute:

In [32]:
optimal_design, init_perf, final_perf, final_iter, ubalance_ratio = design.optimise(init_design=init_design,V=V,model='mnl',time_lim = 1, verbose = True)

Evaluating initial design
Optimization complete 0:00:59 / D-error: 0.206886
Elapsed time: 0:01:00
D-error of initial design:  0.397002
D-error of last stored design:  0.206886
Utility Balance ratio:  93.79 %
Algorithm iterations:  19820



Lastly, the optimal design can be printed:

In [33]:
optimal_design

,CS,alt1_A,alt1_B,alt1_C,alt1_D,alt2_A,alt2_B,alt2_C,alt2_D
0,1,2,2,3,2,1,3,3,0
1,2,3,3,0,1,2,1,5,1
2,3,1,3,5,0,3,2,0,2
3,4,3,1,3,2,2,2,0,0
4,5,2,1,0,0,1,2,3,2
5,6,1,2,5,0,3,1,0,2
6,7,1,2,0,1,3,1,5,1
7,8,2,2,5,2,1,1,0,0
8,9,3,3,5,0,1,1,0,2
9,10,2,3,3,2,1,2,3,0


## (optional) Evaluate the design
The method `evaluate()` allows to evaluate a design stored in a data frame, under the specification provided when `EffDesign` was initialised. `evaluate()` requires the following parameters:

* `optimal_design`: The objective design matrix to evaluate
* `V`: The dictionary object with utility functions
* `model`: The base model of the efficient design. By default is `mnl` for a Multinomial Logit model.

In [34]:
perf, ubalance = design.evaluate(optimal_design,V,model='mnl')

print(perf, ubalance)

0.20688557253691944 93.7905073195354


# References

[1] Quan, W., Rose, J. M., Collins, A. T., & Bliemer, M. C. (2011). A comparison of algorithms for generating efficient choice experiments.

